In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

In [ ]:
from moscowhack.backend.postgres import get_postgres_connection, SkolkovoProm

In [ ]:
psql_db = get_postgres_connection()

In [ ]:
psql_db.bind([SkolkovoProm])

In [ ]:
with psql_db:
    raw_select = SkolkovoProm.select()
    records = list(raw_select.dicts())

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

In [ ]:
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, max_length=2048, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()


In [ ]:
embeds = {
    record['inn']:
    embed_bert_cls(
        record['description'].replace("['", "").replace("']", '')
        model, tokenizer
    )
    for record in records
}
